In [24]:
import pandas as pd
from pathlib import Path
from src.data import CENTERS
from src.config import lesion_level_labels_csv, dmtr_csv

r = Path('/mnt/c/Users/user/data/results_dl')
df = pd.concat([
    pd.read_csv(r / c / 'dl_preds.csv') for c in CENTERS
]).set_index('Unnamed: 0')

df['dl_preds'] = df.mean(axis=1)
lesion_fu = pd.read_csv(lesion_level_labels_csv, sep=';').set_index('lesion')
dmtr = pd.read_csv(dmtr_csv).set_index('id')

df = df.merge(lesion_fu[['patient']], left_index=True, right_index=True)
df = df.groupby('patient').dl_preds.min().to_frame()

preds = pd.concat([
    pd.read_csv(r / 'radiomics_comparison' / c / 'test_outputs.csv') for c in CENTERS
]).set_index('id')

preds = preds.join(df).dropna()
preds = preds.join(dmtr[['response','dcb','center']])

clinical_predictors = pd.read_csv('/mnt/c/Users/user/data/tables/clinical_predictors.csv').set_index('id').drop(columns=['response','center'])

preds = preds.join(clinical_predictors)

/tmp/ipykernel_8136/3219214011.py:13: DtypeWarning: Columns (27,29,175,234,249,259,275,281,313,318,351,393,440,538,547,550,578,585,591,654,660,662,663,664,666,667,669,670,671,676,677,679,680,681,682,685,794,797,805,808,809,818,819,821,822,823,828,851,852,854,855,856,863,864,866,867,868,869,870,872,873,874,875,876,878,879,880,881,882,884,885,886,899,900,902,903,904,905,906,908,909,910,911,912,914,915,916,917,918,920,921,922,924) have mixed types. Specify dtype option on import or set low_memory=False.
  dmtr = pd.read_csv(dmtr_csv).set_index('id')


In [47]:
lesion_fu

,Unnamed: 0,patient,name,center,baseline,fu1,fu2,fu3,delta1,delta2,delta3,last_fu,lesion_benefit,lesion_response,liver,lung,organ,fold
lesion,,,,,,,,,,,,,,,,,,
IM_001_0.nii.gz,0,IM_001,LungMeta01,umcu,6.0,5.0,0.0,0.0,8.333.333.333,0,0,0,1.0,1.0,0.0,1.0,lung,1
IM_001_1.nii.gz,1,IM_001,LungMeta02,umcu,3.6,2.0,0.0,0.0,5.555.555.556,0,0,0,1.0,1.0,0.0,1.0,lung,1
IM_001_2.nii.gz,2,IM_001,LungMeta03,umcu,3.6,2.7,2.5,0.0,75,6.944.444.444,0,0,1.0,1.0,0.0,1.0,lung,1
IM_001_3.nii.gz,3,IM_001,LungMeta04,umcu,3.7,3.3,2.2,0.0,8.918.918.919,5.945.945.946,0,0,1.0,1.0,0.0,1.0,lung,1
IM_001_4.nii.gz,4,IM_001,LiverMeta01,umcu,8.3,8.3,0.0,0.0,100,0,0,0,1.0,1.0,1.0,0.0,liver,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UMCG_738abdomen_0.nii.gz,2767,UMCG_738,subcutis1,umcg,10.0,13.0,12.0,9.0,1.3,1.2,0.9,0.9,1.0,0.0,NaN,NaN,subcutis,2
UMCG_738abdomen_1.nii.gz,2768,UMCG_738,lymph1,umcg,21.0,21.0,0.0,NaN,1,0,NaN,0,1.0,1.0,NaN,NaN,lymph,2
UMCG_738abdomen_2.nii.gz,2769,UMCG_738,lymph2,umcg,19.0,18.0,0.0,NaN,0.947368421,0,NaN,0,1.0,1.0,NaN,NaN,lymph,2
